In [ ]:
import numpy as np
import os
import shutil

In [ ]:
!ls /home/transwarp/kwq/dog_vs_cat/data/train


# 数据预处理

In [24]:
train_dir = '/home/transwarp/kwq/dog_vs_cat/data/train/'
test_dir = '/home/transwarp/kwq/dog_vs_cat/data/test/'

train_dir2 = '/home/transwarp/kwq/dog_vs_cat/data/train2/'
test_dir2 = '/home/transwarp/kwq/dog_vs_cat/data/test2/'

train_filenames = os.listdir(train_dir)
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)


def re_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)
    
re_mkdir(train_dir2)
os.mkdir(train_dir2 + '/dog')
os.mkdir(train_dir2 + '/cat')

re_mkdir(test_dir2)
os.symlink(test_dir, test_dir2 + '/test')

for filename in train_cat:
    os.symlink(train_dir + filename, train_dir2 + '/cat/' + filename)
    
for filename in train_dog:
    os.symlink(train_dir + filename, train_dir2 + '/dog/' + filename)

In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)

    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(train_dir2, image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory(test_dir2, image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)

    train = model.predict_generator(train_generator)
    test = model.predict_generator(test_generator)
    with h5py.File("gap_%s.h5"%MODEL.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

write_gap(ResNet50, (224, 224))
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)

/home/transwarp/python/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [32]:
ResNet50.__

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [29]:
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_dir2, (224, 224), shuffle=False, 
                                             batch_size=16, class_mode=None)

Found 12500 images belonging to 1 classes.


In [ ]:
test_generator.n